In [7]:
import random
import numpy as np
import torch

In [8]:
# 设置随机种子
torch.manual_seed(2)
Weight = torch.randn((10,12)) # d=10, k=12
print(f"Weight.size: {Weight.size()}")
print(f"{Weight}") # d x k=10 x 12

Weight.size: torch.Size([10, 12])
tensor([[-1.0408,  0.9166, -1.3042, -1.1097, -1.2188,  1.1676, -1.0574, -0.1188,
         -0.9078,  0.3452, -0.5713, -0.2351],
        [ 1.0076, -0.7529, -0.2250, -0.4327, -1.5071, -0.4586, -0.8480,  0.5266,
          0.0299, -0.0498,  1.0651,  0.8860],
        [ 0.4640, -0.4986,  0.1289,  2.7631,  0.1405,  1.1191,  0.3152,  1.7528,
         -0.7650,  1.8299, -0.2784, -0.2719],
        [-1.2944, -0.0243, -0.2354, -0.7087,  1.1566,  0.4296, -1.1874, -0.7468,
         -0.9320, -0.8579, -0.9647, -0.0991],
        [-1.0190,  0.3157, -1.6036,  1.8493,  0.0447,  1.5853, -0.5912,  1.1312,
          0.7562, -1.2023, -0.5833, -0.4407],
        [-1.9791,  0.7787, -0.7749, -0.1398,  1.1414, -0.6354, -1.4702, -0.2134,
         -0.8707,  1.6159, -0.2356,  0.9444],
        [ 2.1134, -0.9754,  0.1757, -0.1319, -0.2735,  0.3355,  0.1885,  2.1432,
          0.8527,  0.0965, -0.0625,  0.8269],
        [ 0.5599, -0.7776,  0.3339,  0.1759,  2.1108,  1.0702,  0.0195,  1.12

# decompose

## magnitude

In [5]:
m = torch.linalg.norm(Weight,dim=0).unsqueeze(0)
print(f"m.size: {m.size()}") # k=12 
print(f"{m}") # 1 x k = 1 x 12

m.size: torch.Size([1, 12])
tensor([[3.7444, 2.0768, 2.2998, 3.6691, 3.4267, 2.9005, 2.4539, 3.8735, 3.7141,
         3.2475, 2.1646, 3.1848]])


In [19]:
print(1/m)
print(1/m * Weight)
print(m * 1/m * Weight == Weight)

tensor([[0.2671, 0.4815, 0.4348, 0.2725, 0.2918, 0.3448, 0.4075, 0.2582, 0.2692,
         0.3079, 0.4620, 0.3140]])
tensor([[-0.2780,  0.4413, -0.5671, -0.3024, -0.3557,  0.4025, -0.4309, -0.0307,
         -0.2444,  0.1063, -0.2639, -0.0738],
        [ 0.2691, -0.3625, -0.0978, -0.1179, -0.4398, -0.1581, -0.3456,  0.1359,
          0.0081, -0.0153,  0.4920,  0.2782],
        [ 0.1239, -0.2401,  0.0560,  0.7531,  0.0410,  0.3858,  0.1285,  0.4525,
         -0.2060,  0.5635, -0.1286, -0.0854],
        [-0.3457, -0.0117, -0.1023, -0.1932,  0.3375,  0.1481, -0.4839, -0.1928,
         -0.2509, -0.2642, -0.4457, -0.0311],
        [-0.2721,  0.1520, -0.6973,  0.5040,  0.0131,  0.5465, -0.2409,  0.2920,
          0.2036, -0.3702, -0.2695, -0.1384],
        [-0.5286,  0.3750, -0.3369, -0.0381,  0.3331, -0.2191, -0.5991, -0.0551,
         -0.2344,  0.4976, -0.1089,  0.2965],
        [ 0.5644, -0.4697,  0.0764, -0.0359, -0.0798,  0.1157,  0.0768,  0.5533,
          0.2296,  0.0297, -0.0289,  0.25

# Adapt

测试前先假设存在A，B两个矩阵构成Delta_W

In [10]:
# Delta_W
torch.manual_seed(6)
r = 2
A = torch.randn((10,2))
torch.manual_seed(9)
B = torch.randn((2,12))
Delta_W = A@B
print(f"A.size: {A.size()}; B.size: {B.size()}")
print(f"Delta_W.size: {Delta_W.size()}")
print(f"Delta_W: {Delta_W}")

A.size: torch.Size([10, 2]); B.size: torch.Size([2, 12])
Delta_W.size: torch.Size([10, 12])
Delta_W: tensor([[ 1.3302,  0.1306, -0.9362,  0.9736, -2.1894, -0.9726, -0.4622,  3.5969,
          0.6664, -0.6137, -0.3908,  1.8436],
        [ 1.4915,  2.6190, -2.4163,  4.2948, -1.2002, -0.4393,  4.1025,  2.3347,
         -4.7931,  0.6530,  2.1819,  1.9760],
        [-0.1989,  0.7557, -0.2885,  0.8587,  0.7207,  0.3496,  1.5179, -1.0703,
         -1.8367,  0.5122,  0.8799, -0.3042],
        [ 1.1632, -1.4600,  0.0513, -1.1878, -2.7133, -1.2651, -3.3460,  4.2267,
          4.1098, -1.3904, -2.0098,  1.6702],
        [-1.0529, -1.7084,  1.6281, -2.8499,  0.9184,  0.3471, -2.6337, -1.7444,
          3.0690, -0.3848, -1.3914, -1.4000],
        [-0.1635,  0.6447, -0.2501,  0.7363,  0.6043,  0.2935,  1.2916, -0.8959,
         -1.5624,  0.4338,  0.7482, -0.2509],
        [-0.5991, -0.8337,  0.8499, -1.4423,  0.5928,  0.2339, -1.2400, -1.0876,
          1.4362, -0.1439, -0.6451, -0.8017],
        [ 

得到 W_new = W+Delta_W

In [13]:
W_new = Weight + Delta_W
print(f"W_new: {W_new}")

W_new: tensor([[ 0.2894,  1.0472, -2.2404, -0.1361, -3.4081,  0.1950, -1.5196,  3.4781,
         -0.2415, -0.2685, -0.9622,  1.6085],
        [ 2.4991,  1.8661, -2.6413,  3.8621, -2.7073, -0.8979,  3.2545,  2.8613,
         -4.7632,  0.6031,  3.2469,  2.8620],
        [ 0.2651,  0.2571, -0.1596,  3.6218,  0.8612,  1.4687,  1.8331,  0.6824,
         -2.6016,  2.3421,  0.6015, -0.5762],
        [-0.1312, -1.4843, -0.1840, -1.8965, -1.5567, -0.8355, -4.5333,  3.4799,
          3.1778, -2.2483, -2.9745,  1.5711],
        [-2.0718, -1.3927,  0.0245, -1.0005,  0.9631,  1.9323, -3.2249, -0.6132,
          3.8252, -1.5871, -1.9748, -1.8407],
        [-2.1426,  1.4234, -1.0250,  0.5966,  1.7457, -0.3418, -0.1786, -1.1093,
         -2.4331,  2.0497,  0.5126,  0.6935],
        [ 1.5143, -1.8091,  1.0256, -1.5742,  0.3193,  0.5694, -1.0515,  1.0556,
          2.2889, -0.0474, -0.7076,  0.0251],
        [ 0.5665, -1.2597,  0.5961, -0.4447,  1.8549,  0.9382, -0.8850,  1.4708,
         -0.4022, -0.46

In [15]:
M = torch.linalg.norm(W_new,dim=0).unsqueeze(0)
print(f"M.size: {M.size()}")
print(f"{M}")
print(f"1/M: {1/M}")

M.size: torch.Size([1, 12])
tensor([[4.3794, 5.0311, 4.3672, 7.1422, 5.5332, 3.5157, 8.5034, 7.4857, 9.2414,
         4.5263, 5.5304, 4.6274]])
1/M: tensor([[0.2283, 0.1988, 0.2290, 0.1400, 0.1807, 0.2844, 0.1176, 0.1336, 0.1082,
         0.2209, 0.1808, 0.2161]])


用原来的m去merge一下看看

In [17]:
print(f"{m * 1/M *W_new}")

tensor([[ 0.2474,  0.4323, -1.1798, -0.0699, -2.1107,  0.1609, -0.4385,  1.7998,
         -0.0970, -0.1926, -0.3766,  1.1070],
        [ 2.1368,  0.7703, -1.3909,  1.9841, -1.6767, -0.7408,  0.9392,  1.4806,
         -1.9143,  0.4327,  1.2709,  1.9698],
        [ 0.2267,  0.1061, -0.0841,  1.8606,  0.5334,  1.2117,  0.5290,  0.3531,
         -1.0456,  1.6804,  0.2354, -0.3966],
        [-0.1122, -0.6127, -0.0969, -0.9743, -0.9641, -0.6893, -1.3082,  1.8007,
          1.2771, -1.6131, -1.1643,  1.0813],
        [-1.7714, -0.5749,  0.0129, -0.5140,  0.5965,  1.5942, -0.9306, -0.3173,
          1.5373, -1.1387, -0.7729, -1.2669],
        [-1.8319,  0.5876, -0.5398,  0.3065,  1.0811, -0.2820, -0.0515, -0.5740,
         -0.9778,  1.4706,  0.2006,  0.4773],
        [ 1.2947, -0.7468,  0.5401, -0.8087,  0.1977,  0.4698, -0.3034,  0.5462,
          0.9199, -0.0340, -0.2770,  0.0173],
        [ 0.4844, -0.5200,  0.3139, -0.2284,  1.1488,  0.7741, -0.2554,  0.7611,
         -0.1616, -0.3367, -0.

## 事实上，作者将m设置为trainable的部分，因此由模型训练来让m逼近期望的M。

# ================================================

In [13]:
import torch
# 使用 torch.linalg.svd 进行奇异值分解
U, S, Vh = torch.linalg.svd(Weight)

# Vh 是 V 的转置，需要转置回来
V = Vh.t()

# 打印分解结果
print("矩阵 W:")
print(Weight)
print("\n左奇异向量 U:")
print(U.size(), U)
print("\n奇异值 S:")
print(S.size(), S)
print("\n右奇异向量 Vh:")
print(V.size(), V)

# 重构原始矩阵
# 使用 torch.diag_embed 将 S 变成对角矩阵，再进行矩阵乘法
S_diag = torch.diag_embed(S)
print(S_diag.size())
A_reconstructed = torch.mm(U, torch.mm(S_diag, V.t()))
print("\n重构的矩阵 Weight:")
print(A_reconstructed)

# 检查原矩阵与重构矩阵之间的差异
difference = torch.norm(Weight - A_reconstructed)
print("\n原矩阵与重构矩阵之间的差异:")
print(difference)

矩阵 W:
tensor([[-1.0408,  0.9166, -1.3042, -1.1097, -1.2188,  1.1676, -1.0574, -0.1188,
         -0.9078,  0.3452, -0.5713, -0.2351],
        [ 1.0076, -0.7529, -0.2250, -0.4327, -1.5071, -0.4586, -0.8480,  0.5266,
          0.0299, -0.0498,  1.0651,  0.8860],
        [ 0.4640, -0.4986,  0.1289,  2.7631,  0.1405,  1.1191,  0.3152,  1.7528,
         -0.7650,  1.8299, -0.2784, -0.2719],
        [-1.2944, -0.0243, -0.2354, -0.7087,  1.1566,  0.4296, -1.1874, -0.7468,
         -0.9320, -0.8579, -0.9647, -0.0991],
        [-1.0190,  0.3157, -1.6036,  1.8493,  0.0447,  1.5853, -0.5912,  1.1312,
          0.7562, -1.2023, -0.5833, -0.4407],
        [-1.9791,  0.7787, -0.7749, -0.1398,  1.1414, -0.6354, -1.4702, -0.2134,
         -0.8707,  1.6159, -0.2356,  0.9444],
        [ 2.1134, -0.9754,  0.1757, -0.1319, -0.2735,  0.3355,  0.1885,  2.1432,
          0.8527,  0.0965, -0.0625,  0.8269],
        [ 0.5599, -0.7776,  0.3339,  0.1759,  2.1108,  1.0702,  0.0195,  1.1213,
         -1.4873, -0.204

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x10 and 12x12)